In [1]:
!pip install -U tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/ba/7c/b971f2485155917ecdcebb210e021e36a6b65457394590be01cc61515310/tensorflow-2.13.0-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.13.0 from https://files.pythonhosted.org/packages/40/fa/98115f6fe4d92e1962f549917be2dc8e369853b7e404191996fedaaf4dd6/tensorflow_intel-2.13.0-cp310-cp310-win_amd64.whl.metadata
     ---------------------------------------- 0.0/126.5 kB ? eta -:--:--
     -------------------------------------- 126.5/126.5 kB 7.8 MB/s eta 0:00:00
  Obtaining dependency information for flatbuffers>=23.1.21 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/57.5 kB ? eta -:--:--
     ---------------------------------------- 57.5/57.5 kB ? eta 0:00:00
  Obtaining dependency informa

In [2]:
import tensorflow as tf
print("TensorFlow:", tf.__version__)

TensorFlow: 2.13.0


## 텐서 와 연산

In [5]:
#스칼라 텐서를 생성
t1 = tf.constant(42)
print(t1)
#배열을 가지고 생성
t2 = tf.constant([[1, 2, 3], [4, 5, 6]])
print(t2)

tf.Tensor(42, shape=(), dtype=int32)
tf.Tensor(
[[1 2 3]
 [4 5 6]], shape=(2, 3), dtype=int32)


In [6]:
#인덱싱
print(t2[:, 1:]) #행전체 그리고 열은 1번 부터 끝까지 선택
print(t2[:, 1]) #첫번째 열만 선택 - 일차원 배열로 생성
print(t2[:, 1, tf.newaxis]) #첫번째 열만 선택 - 다차원 배열로 생성
#tf.newaxis를 추가하면 차원이 하나 추가됩니다.

tf.Tensor(
[[2 3]
 [5 6]], shape=(2, 2), dtype=int32)
tf.Tensor([2 5], shape=(2,), dtype=int32)
tf.Tensor(
[[2]
 [5]], shape=(2, 1), dtype=int32)


In [7]:
#연산
print(t2 + 10) #broadcast 연산 수행
print(tf.square(t2))
print(t2 @ tf.transpose(t2))

tf.Tensor(
[[11 12 13]
 [14 15 16]], shape=(2, 3), dtype=int32)
tf.Tensor(
[[ 1  4  9]
 [16 25 36]], shape=(2, 3), dtype=int32)
tf.Tensor(
[[14 32]
 [32 77]], shape=(2, 2), dtype=int32)


In [8]:
from tensorflow import keras
K = keras.backend
K.square(K.transpose(t2)) + 10

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[11, 26],
       [14, 35],
       [19, 46]])>

In [12]:
import numpy as np
ar = np.array([2, 3, 4])
print(np.sum(ar))

#numpy 배열을 가지고 Tensor 생성
x = tf.constant(ar)
print(x)
#Tensor를 numpy 배열로 변경
print(type(x.numpy()))

#numpy 와 Tensor 연산에 데이터 공유 가능
print(tf.square(ar))
print(np.square(x))

9
tf.Tensor([2 3 4], shape=(3,), dtype=int32)
<class 'numpy.ndarray'>
tf.Tensor([ 4  9 16], shape=(3,), dtype=int32)
[ 4  9 16]


In [15]:
# 자료형이 달라서 에러 발생
try:
    tf.constant(10) + tf.constant(10.3)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

cannot compute AddV2 as input #1(zero-based) was expected to be a int32 tensor but is a float tensor [Op:AddV2] name: 


In [16]:
try:
    print(tf.cast(tf.constant(10), tf.float32) + tf.constant(10.3))
except tf.errors.InvalidArgumentError as ex:
    print(ex)

tf.Tensor(20.3, shape=(), dtype=float32)


In [18]:
#변수 생성
v = tf.Variable([[1, 2, 3], [4, 5, 6]])
print(v)
v.assign(5 + v)
print(v)
v[0,0].assign(42)
print(v)

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>
<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[ 6,  7,  8],
       [ 9, 10, 11]])>
<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[42,  7,  8],
       [ 9, 10, 11]])>


## 문자열 Tensor

In [19]:
tf.constant("안녕하세요") #하나의 기본 자료형으로 인코딩 해서 저장


<tf.Tensor: shape=(), dtype=string, numpy=b'\xec\x95\x88\xeb\x85\x95\xed\x95\x98\xec\x84\xb8\xec\x9a\x94'>

In [26]:
#Tensor 배열 - 실수 3개를 저장하는 텐서 배열을 생성
array = tf.TensorArray(dtype=tf.float32, size=3)
#3개의 데이터 저장
array = array.write(0, tf.constant([1.0, 2.0]))
array = array.write(1, tf.constant([3.0, 4.0]))
array = array.write(2, tf.constant([5.0, 6.0]))
#첫번째 데이터 읽기
print(array.read(1))

tf.Tensor([3. 4.], shape=(2,), dtype=float32)


In [36]:
#ragged tensor: 각 배열의 길이가 달라도 됩니다.
ar = tf.ragged.constant([[10, 20], [30, 40, 50], [50, 60]])
print(ar)

<tf.RaggedTensor [[10.0, 20.0], [30.0, 40.0, 50.0], [50.0, 60.0]]>


In [38]:
#희소 텐서 생성
#3행 4열의 행렬을 만들고 indices 자리에 values의 값을 순서대로 배정한
#희소 행렬
s = tf.SparseTensor(indices=[[0,1], [1, 0], [2, 3]],
                             values = [1.0, 2.0, 3.0],
                    dense_shape=[3, 4])
print(tf.sparse.to_dense(s))

tf.Tensor(
[[0. 1. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


In [28]:
s1 = tf.constant([[1, 2, 3, 1, 1], [2, 4, 5, 8, 11]])
s2 = tf.constant([[1, 2, 3, 4, 4], [2, 7, 5, 8, 9]])
print(s1)
print(s2)
print(tf.sparse.to_dense(tf.sets.union(s1, s2)))

# set 연산은 배열로 리턴하기 때문에 연산의 결과 각 배열의 데이터 개수가 
# 다르면 뒤에 0을 채워 동일한 개수로 맞춤

tf.Tensor(
[[ 1  2  3  1  1]
 [ 2  4  5  8 11]], shape=(2, 5), dtype=int32)
tf.Tensor(
[[1 2 3 4 4]
 [2 7 5 8 9]], shape=(2, 5), dtype=int32)
tf.Tensor(
[[ 1  2  3  4  0  0  0]
 [ 2  4  5  7  8  9 11]], shape=(2, 7), dtype=int32)


In [42]:
#노드 생성
node1 = tf.constant(3.0)
node2 = tf.constant(4.0)

#그래프 실행
print(node1)
print(node2)



tf.Tensor(3.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [33]:
#텐서플로우 코드 최적화가 가능하도록 작성
@tf.function
def tf_cube(x):
  print(x)#값의 변화가 없는 구문이므로 함수 호출전에 전부 수행해버림
  return x ** 3 #텐서플로우 함수 호출 구문으로 변경되서 실행

result = tf_cube(tf.constant(2.0))
print(result)

Tensor("x:0", shape=(), dtype=float32)
tf.Tensor(8.0, shape=(), dtype=float32)


In [34]:
#2행 5열의 난수 발생: 0에서 1사이에서 균등한 분포로
result = tf.random.uniform([2, 5], 0.0, 1.0)
print(result)

tf.Tensor(
[[0.00735152 0.04825318 0.5854404  0.98272777 0.6578027 ]
 [0.14534378 0.16297448 0.8356873  0.6923982  0.6557356 ]], shape=(2, 5), dtype=float32)


In [35]:
#2행 5열의 난수 발생: 0에서 1사이에서 정규 분포에서 평균은 0
#표준 편차는 1
result = tf.random.normal([2, 5], 0.0, 1.0)
print(result)

tf.Tensor(
[[-3.0151565  -0.89438844  2.1415417   0.5931125  -0.9252684 ]
 [-0.49756595  0.65568143  0.28834954 -0.22699893  0.29563606]], shape=(2, 5), dtype=float32)


## 뉴런 생성

In [36]:
# 활성화 함수 - 시그모이드
import math

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [39]:
#입력 값
X = 1.0
#타겟
y = 0.0
#가중치를 랜덤하게 설정
w = tf.random.normal([1], 0, 1)
#출력
output = sigmoid(X * w)
print("예측값:", output)

예측값: 0.40794543534869565


In [41]:
#경사 하강법
for i in range(1000):
  output = sigmoid(X * w)
  error = y - output
  w = w + X * 0.1 * error #학습률을 0.1로 설정
  if i % 50 == 0:
    print(i, output, error)

0 0.010159530042749975 -0.010159530042749975
50 0.009672833393959416 -0.009672833393959416
100 0.009230413229461887 -0.009230413229461887
150 0.00882655169344161 -0.00882655169344161
200 0.00845641934224771 -0.00845641934224771
250 0.008115963652056576 -0.008115963652056576
300 0.007801768554795715 -0.007801768554795715
350 0.007510904622438072 -0.007510904622438072
400 0.007240882980793758 -0.007240882980793758
450 0.006989546391939811 -0.006989546391939811
500 0.006754999103848625 -0.006754999103848625
550 0.006535650736737556 -0.006535650736737556
600 0.006330058643178731 -0.006330058643178731
650 0.006136963651753118 -0.006136963651753118
700 0.005955279139837703 -0.005955279139837703
750 0.0057840154921772165 -0.0057840154921772165
800 0.005622301918124851 -0.005622301918124851
850 0.005469360191126603 -0.005469360191126603
900 0.00532449985249662 -0.00532449985249662
950 0.005187096630971439 -0.005187096630971439


In [43]:
#경사 하강법 문제 발생
#가중치 만을 곱하게 되는 경우 Input이 0이면 가중치가 변경되지 않음
X = 0
y = 1
w = tf.random.normal([1], 0, 1)
for i in range(1000):
  output = sigmoid(X * w)
  error = y - output
  w = w + X * 0.1 * error #학습률을 0.1로 설정
  if i % 50 == 0:
    print(i, output, error)

0 0.5 0.5
50 0.5 0.5
100 0.5 0.5
150 0.5 0.5
200 0.5 0.5
250 0.5 0.5
300 0.5 0.5
350 0.5 0.5
400 0.5 0.5
450 0.5 0.5
500 0.5 0.5
550 0.5 0.5
600 0.5 0.5
650 0.5 0.5
700 0.5 0.5
750 0.5 0.5
800 0.5 0.5
850 0.5 0.5
900 0.5 0.5
950 0.5 0.5


In [60]:
#경사 하강법 문제 발생 - 편향을 추가해서 해결
X = 0
y = 1
w = tf.random.normal([1], 0, 1)

b = tf.random.normal([1], 0, 1)
for i in range(1000):
  output = sigmoid(X * w + 1 * b)
  error = y - output
  w = w + X * 0.1 * error #학습률을 0.1로 설정
  b = b + 1 * 0.1 * error
  if i % 50 == 0:
    print(i, output, error)

0 0.19769055719411938 0.8023094428058806
50 0.7405050458806913 0.2594949541193087
100 0.8749037773548664 0.12509622264513365
150 0.9201117781390925 0.0798882218609075
200 0.9418358007999427 0.05816419920005733
250 0.9544366851244958 0.045563314875504246
300 0.9626181063519526 0.03738189364804745
350 0.968341525052936 0.031658474947063975
400 0.9725627616916909 0.02743723830830913
450 0.9758011007238457 0.024198899276154262
500 0.9783621276398727 0.021637872360127264
550 0.9804371786102818 0.01956282138971821
600 0.9821519302602834 0.017848069739716643
650 0.9835923354875891 0.016407664512410935
700 0.9848190589327792 0.015180941067220788
750 0.9858761985197096 0.014123801480290354
800 0.986796489112087 0.01320351088791305
850 0.987604820445823 0.012395179554177016
900 0.988320367829414 0.011679632170585985
950 0.9889582222256703 0.011041777774329686


In [44]:
#AND를 뉴런으로 구현
import numpy as np

X = np.array([[0,0], [0,1], [1,0], [1,1]])
y = np.array([[0], [0], [0], [1]])
b = tf.random.normal([1], 0, 1)
#입력이 2개라서 가중치를 2개 생성
w = tf.random.normal([2], 0, 1)

for i in range(2000):
  #오차의 합계를 저장할 변수
  error_sum = 0
  #모든 입력을 대입해서 수행
  for j in range(4):
    output = sigmoid(np.sum(X[j] * w) + 1 * b)
    error = y[j][0] - output
    w = w + X[j] * 0.1 * error
    b = b + 1 * 0.1 * error
    error_sum += error
  if i % 100 == 0:
    print(i, error_sum)

0 0.14899253316491956
100 -0.1676879182578238
200 -0.10922975461116352
300 -0.08170167580681198
400 -0.06528802743620812
500 -0.05431837178820545
600 -0.04645829176636759
700 -0.04055252165688436
800 -0.03595622701241791
900 -0.032279843161930985
1000 -0.02927331049393865
1100 -0.026771249662140215
1200 -0.02465721411853157
1300 -0.022848183395979438
1400 -0.02128315127155317
1500 -0.01991585933091608
1600 -0.018711837377670508
1700 -0.01764317807874973
1800 -0.016687686291351075
1900 -0.015830694655368363


In [45]:
#predict
for i in range(4):
  print("X:", X[i], "Y:", y[i], "Predict:", sigmoid(np.sum(X[i]*w)+b))

X: [0 0] Y: [0] Predict: 2.3874530876990014e-05
X: [0 1] Y: [0] Predict: 0.02506080898494512
X: [1 0] Y: [0] Predict: 0.02513738464225334
X: [1 1] Y: [1] Predict: 0.9652317852804531


### OR를 경사하강법으로 구현

In [46]:
#OR를 뉴런으로 구현
import numpy as np

X = np.array([[0,0], [0,1], [1,0], [1,1]])
y = np.array([[0], [1], [1], [1]])
b = tf.random.normal([1], 0, 1)
#입력이 2개라서 가중치를 2개 생성
w = tf.random.normal([2], 0, 1)

for i in range(2000):
  #오차의 합계를 저장할 변수
  error_sum = 0
  #모든 입력을 대입해서 수행
  for j in range(4):
    output = sigmoid(np.sum(X[j] * w) + 1 * b)
    error = y[j][0] - output
    w = w + X[j] * 0.1 * error
    b = b + 1 * 0.1 * error
    error_sum += error
  if i % 100 == 0:
    print(i, error_sum)

#predict
for i in range(4):
  print("X:", X[i], "Y:", y[i], "Predict:", sigmoid(np.sum(X[i]*w)+b))

0 0.792447094265374
100 -0.09191918438934005
200 -0.05030513393200414
300 -0.03448174634615714
400 -0.026145875284354247
500 -0.02100659078212877
600 -0.01752811554508281
700 -0.015022352944278339
800 -0.013133944067730177
900 -0.01166114553242046
1000 -0.010481282424876413
1100 -0.00951559695262498
1200 -0.008710619053220382
1300 -0.008029887653971973
1400 -0.007447129294170622
1500 -0.0069422272554561446
1600 -0.0065005260599870565
1700 -0.00611143077630729
1800 -0.005766273595097592
1900 -0.005456753958207532
X: [0 0] Y: [0] Predict: 0.025687934543840916
X: [0 1] Y: [1] Predict: 0.9897817722690809
X: [1 0] Y: [1] Predict: 0.9897533664796261
X: [1 1] Y: [1] Predict: 0.9999971821333941


In [47]:
#XOR를 뉴런으로 구현
import numpy as np

X = np.array([[0,0], [0,1], [1,0], [1,1]])
y = np.array([[0], [1], [1], [0]])
b = tf.random.normal([1], 0, 1)
#입력이 2개라서 가중치를 2개 생성
w = tf.random.normal([2], 0, 1)

for i in range(2000):
  #오차의 합계를 저장할 변수
  error_sum = 0
  #모든 입력을 대입해서 수행
  for j in range(4):
    output = sigmoid(np.sum(X[j] * w) + 1 * b)
    error = y[j][0] - output
    w = w + X[j] * 0.1 * error
    b = b + 1 * 0.1 * error
    error_sum += error
  if i % 100 == 0:
    print(i, error_sum)

#predict
for i in range(4):
  print("X:", X[i], "Y:", y[i], "Predict:", sigmoid(np.sum(X[i]*w)+b))

0 -0.8321403412102342
100 0.008287363923597746
200 0.0016700375060174633
300 0.0003367578312285513
400 6.7920350119266e-05
500 1.3693577613604901e-05
600 2.755835269763196e-06
700 5.574956685183707e-07
800 1.1913095121496298e-07
900 4.746623760709667e-08
1000 4.746623760709667e-08
1100 4.746623760709667e-08
1200 4.746623760709667e-08
1300 4.746623760709667e-08
1400 4.746623760709667e-08
1500 4.746623760709667e-08
1600 4.746623760709667e-08
1700 4.746623760709667e-08
1800 4.746623760709667e-08
1900 4.746623760709667e-08
X: [0 0] Y: [0] Predict: 0.5128175691057347
X: [0 1] Y: [1] Predict: 0.49999997485429043
X: [1 0] Y: [1] Predict: 0.4871823676059484
X: [1 1] Y: [0] Predict: 0.47438160794816525


In [49]:
X = np.array([[0,0], [0,1], [1,0], [1,1]])
y = np.array([[0], [1], [1], [0]])

model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=2, activation='sigmoid',
                          input_shape=(2,)),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.01), loss='mse')

model.summary()

history = model.fit(X, y, epochs=1000, batch_size=1)

model.predict(X)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 2)                 6         
                                                                 
 dense_3 (Dense)             (None, 1)                 3         
                                                                 
Total params: 9 (36.00 Byte)
Trainable params: 9 (36.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.2519
Epoch 2/1000
4/4 [==============================] - 0s 1000us/step - loss: 0.2519
Epoch 3/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.2518
Epoch 4/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.2518
Epoch 5/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.2518
Epoch 6/1000
4/4 [==============================] - 0s 2ms/step - loss: 0.2518
Epoch 7/1000
4/4 [==============================] - 0s 0s/step - loss: 0.2518
Epoch 8/1000
4/4 [==============================] - 0s 944us/step - loss: 0.2518
Epoch 9/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.2517
Epoch 10/1000
4/4 [==============================] - 0s 524us/step - loss: 0.2517
Epoch 11/1000
4/4 [==============================] - 0s 3ms/step - loss: 0.2517
Epoch 12/1000
4/4 [==============================] - 0s 597us/step - loss: 0.2517
Epoch 13/1000
4/4 [======================

array([[0.5039279 ],
       [0.49855673],
       [0.50284636],
       [0.4979948 ]], dtype=float32)